## Задание

Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:

* x - последовательность цифр
* y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
* научить модель предсказывать y(i) по x(i)
* пробовать RNN, LSTM, GRU

In [1]:
import torch
import pandas as pd
import time

In [2]:
def generate_x(size):
    X = torch.Tensor()
    """Функция генерирует последовательность X размерности sizes в соответствии с заданными условиями.
    size: tuple"""
    X = torch.randint(0, 10, (size, ))
    return X

In [3]:
sizes = (50, 200, 500)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X = generate_x(sizes[0])
X

tensor([5, 6, 0, 6, 1, 8, 0, 8, 7, 0, 7, 8, 5, 3, 7, 5, 9, 0, 0, 2, 2, 1, 2, 3,
        5, 2, 6, 7, 4, 5, 1, 5, 4, 2, 1, 3, 1, 3, 3, 7, 7, 0, 6, 3, 5, 3, 8, 9,
        3, 3])

In [4]:
def generate_y(X, size):
    """Функция генерирует последовательность X размерности size в соответствии с заданными условиями.
    X: tensor,
    size: tuple"""
    Y = []
    for i in range(len(X)):
        if i == 0:
            Y.append(int(X[0]))
        else:
            num = X[i] + X[0]
            while num >= 10:
                num -= 10
            Y.append(int(num))
    Y = torch.Tensor(Y)
    return Y

In [5]:
Y = generate_y(X, sizes[0])
Y

tensor([5., 1., 5., 1., 6., 3., 5., 3., 2., 5., 2., 3., 0., 8., 2., 0., 4., 5.,
        5., 7., 7., 6., 7., 8., 0., 7., 1., 2., 9., 0., 6., 0., 9., 7., 6., 8.,
        6., 8., 8., 2., 2., 5., 1., 8., 0., 8., 3., 4., 8., 8.])

In [6]:
class Network(torch.nn.Module):
    def __init__(self, typeRNN, size):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(10, size)
        self.rnn = typeRNN(size, 128)
        self.out = torch.nn.Linear(128, 10)

    def forward(self, seq):
        x = self.embedding(seq)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

In [8]:
def train(epochs, num_batches, sizes, device):
    rnns = [torch.nn.RNN, torch.nn.LSTM, torch.nn.GRU]
    models = dict()
    for rnn in rnns:
        model_size = dict()
        for size in sizes:
            model = Network(rnn, size).to(device)
            start = time.time()
            criterion = torch.nn.CrossEntropyLoss().to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=.05)

            for ep in range(epochs):
                train_loss = 0.
            
                for i in range(num_batches):
                    X_batch = generate_x(size).to(device)
                    Y_batch = generate_y(X_batch, size).to(device).long()
            
                    optimizer.zero_grad()
                    answers = model.forward(X_batch).to(device)

                    loss = criterion(answers, Y_batch)
                    train_loss += loss.item()
            
                    loss.backward()
                    optimizer.step()
            
            print(f"RNN: {rnn}. Size {size}. Time: {time.time()-start}, Train loss: {train_loss}")
            model_size[size] = train_loss
            
        models[rnn] = model_size

    return models

In [9]:
res = train(epochs=5, num_batches=10, sizes=sizes, device=device)

RNN: <class 'torch.nn.modules.rnn.RNN'>. Size 50. Time: 1.8914756774902344, Train loss: 31.356383562088013
RNN: <class 'torch.nn.modules.rnn.RNN'>. Size 200. Time: 2.7056398391723633, Train loss: 27.963059902191162
RNN: <class 'torch.nn.modules.rnn.RNN'>. Size 500. Time: 6.836570739746094, Train loss: 49.610647201538086
RNN: <class 'torch.nn.modules.rnn.LSTM'>. Size 50. Time: 0.7887477874755859, Train loss: 24.86144995689392
RNN: <class 'torch.nn.modules.rnn.LSTM'>. Size 200. Time: 2.7475106716156006, Train loss: 22.61834156513214
RNN: <class 'torch.nn.modules.rnn.LSTM'>. Size 500. Time: 7.223398447036743, Train loss: 25.285171508789062
RNN: <class 'torch.nn.modules.rnn.GRU'>. Size 50. Time: 0.7845098972320557, Train loss: 27.78986144065857
RNN: <class 'torch.nn.modules.rnn.GRU'>. Size 200. Time: 2.9211976528167725, Train loss: 29.319033980369568
RNN: <class 'torch.nn.modules.rnn.GRU'>. Size 500. Time: 6.988177537918091, Train loss: 32.38724100589752


In [13]:
pd.DataFrame(res)

,<class 'torch.nn.modules.rnn.RNN'>,<class 'torch.nn.modules.rnn.LSTM'>,<class 'torch.nn.modules.rnn.GRU'>
50,31.356384,24.861450,27.789861
200,27.963060,22.618342,29.319034
500,49.610647,25.285172,32.387241


Полученные данные свидетельствуют о том, что на длинных последовательностях лучший результат показывает LSTM, а худший - RNN, за счет "забывания".